In [1]:
## the file path

import pickle, sys
import numpy as np
from scipy.spatial import distance
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.decomposition import TruncatedSVD
import gensim.models.keyedvectors as word2vec


    

    
def create_weightedPCAremoved(dictionary, des):
    
    
    corpus_pca_removed1_embed = []
    corpus_pca_removed3_embed = []
    corpus_pca_removed5_embed = []
    citant_pca_removed1_embed = []
    citant_pca_removed3_embed = []
    citant_pca_removed5_embed = []
    
    count = 0
    
    for keys in dictionary.keys():
#         if count < 2:
            count+=1
            
            ## embedding vectors of corpus and citant sentences
        
            corpus_sentence_embed_arr = dictionary[keys]['corpus_embed']
            citance_sentence_embed_arr = dictionary[keys]['cite_text_embed'] # 18 x 768
            print("\n count :", count)
            print("\n corpus_sentence_embed_arr shape ", corpus_sentence_embed_arr.shape)
            print("\n citance_sentence_embed_arr shape", citance_sentence_embed_arr.shape)

            svd3_corpus = TruncatedSVD(n_components=3, random_state=0).fit(corpus_sentence_embed_arr)
            svd3_citant = TruncatedSVD(n_components=3, random_state=0).fit(citance_sentence_embed_arr)

            svd5_corpus = TruncatedSVD(n_components=5, random_state=0).fit(corpus_sentence_embed_arr)
            svd5_citant = TruncatedSVD(n_components=5, random_state=0).fit(citance_sentence_embed_arr)

            ## normalised eigen values to find weighted projections on eigen vectors/ principal directions
            svd3_corpus_singular_values = svd3_corpus.singular_values_/svd3_corpus.singular_values_.sum()
            svd5_corpus_singular_values = svd5_corpus.singular_values_/svd5_corpus.singular_values_.sum()

            svd3_citant_singular_values = svd3_citant.singular_values_/svd3_citant.singular_values_.sum()
            svd5_citant_singular_values = svd5_citant.singular_values_/svd5_citant.singular_values_.sum()

            
            ## principal directions for corpus and citant sentences
            svd3_corpus_comp = svd3_corpus.components_ # 3 x 768
            svd5_corpus_comp = svd5_corpus.components_ # 5 x 768

            svd3_citant_comp = svd3_citant.components_
            svd5_citant_comp = svd5_citant.components_

#             print("\n svd3_corpus_comp shape",svd3_corpus_comp.shape)
#             print("\n svd5_corpus_comp shape",svd5_corpus_comp.shape)
#             print("\n svd3_citant_comp shape",svd3_citant_comp.shape)
#             print("\n svd5_citant_comp shape",svd5_citant_comp.shape)

            
            
            ## scaler value of embeddings projected on the principal directions. We will multiply
            ## them with the eigen values and Then we will multiply
            ## it with unit eigen vector/ principal directions to find the common component vectors.
            ## then we will add them all. and subtract from the embeddings.
            
            
            ##scaler values
            svd3_corpus_projections = corpus_sentence_embed_arr@svd3_corpus_comp.T #(203, 3) number of sen x no of pc
            svd5_corpus_projections = corpus_sentence_embed_arr@svd5_corpus_comp.T #(203, 5)

            svd3_citant_projections = citance_sentence_embed_arr@svd3_citant_comp.T
            svd5_citant_projections = citance_sentence_embed_arr@svd5_citant_comp.T

#             print("\n svd3_corpus_projections shape",svd3_corpus_projections.shape)
#             print("\n svd5_corpus_projections shape",svd5_corpus_projections.shape)
#             print("\n svd3_citant_projections shape",svd3_citant_projections.shape)
#             print("\n svd5_citant_projections shape",svd5_citant_projections.shape)

            
            ##sclaer values of projections after multiplying with the eigen values (point wise)
            value_weighted_svd3_corpus_projections = svd3_corpus_singular_values * svd3_corpus_projections
            value_weighted_svd5_corpus_projections = svd5_corpus_singular_values * svd5_corpus_projections
            value_weighted_svd3_citant_projections = svd3_citant_singular_values * svd3_citant_projections
            value_weighted_svd5_citant_projections = svd5_citant_singular_values * svd5_citant_projections
            
#             print("\n value_weighted_svd3_corpus_projections shape",value_weighted_svd3_corpus_projections.shape)
#             print("\n value_weighted_svd5_corpus_projections shape",value_weighted_svd5_corpus_projections.shape)
#             print("\n value_weighted_svd3_citant_projections shape",value_weighted_svd3_citant_projections.shape)
#             print("\n value_weighted_svd5_citant_projections shape",value_weighted_svd5_citant_projections.shape)
   
            
    
            ##reshaping
            new_value_weighted_svd3_corpus_projections = value_weighted_svd3_corpus_projections[:,None,:]
            new_value_weighted_svd5_corpus_projections = value_weighted_svd5_corpus_projections[:,None,:]
            new_value_weighted_svd3_citant_projections = value_weighted_svd3_citant_projections[:,None,:]
            new_value_weighted_svd5_citant_projections = value_weighted_svd5_citant_projections[:,None,:]
            
            
            ##reshaping
            new_svd3_corpus_comp = svd3_corpus_comp.T[None,:]
            new_svd5_corpus_comp = svd5_corpus_comp.T[None,:]
            new_svd3_citant_comp = svd3_citant_comp.T[None,:]
            new_svd5_citant_comp = svd5_citant_comp.T[None,:]
            
#             print("\n new_svd3_corpus_comp shape",new_svd3_corpus_comp.shape)
#             print("\n new_svd5_corpus_comp shape",new_svd5_corpus_comp.shape)
#             print("\n new_svd3_citant_comp shape",new_svd3_citant_comp.shape)
#             print("\n new_svd5_citant_comp shape",new_svd5_citant_comp.shape)
   
            
            
            ## multiplying to get vectors
            weighted_projection_vector_svd3_corpus = new_value_weighted_svd3_corpus_projections*new_svd3_corpus_comp
            weighted_projection_vector_svd5_corpus = new_value_weighted_svd5_corpus_projections*new_svd5_corpus_comp
            weighted_projection_vector_svd3_citant = new_value_weighted_svd3_citant_projections*new_svd3_citant_comp
            weighted_projection_vector_svd5_citant = new_value_weighted_svd5_citant_projections*new_svd5_citant_comp
            
            
            sum_weighted_projection_vector_svd3_corpus = weighted_projection_vector_svd3_corpus.sum(axis=2)
            sum_weighted_projection_vector_svd5_corpus = weighted_projection_vector_svd5_corpus.sum(axis=2)
            sum_weighted_projection_vector_svd3_citant = weighted_projection_vector_svd3_citant.sum(axis=2)
            sum_weighted_projection_vector_svd5_citant = weighted_projection_vector_svd5_citant.sum(axis=2)
            
#             print("\n sum_weighted_projection_vector_svd3_corpus shape",sum_weighted_projection_vector_svd3_corpus.shape)
#             print("\n sum_weighted_projection_vector_svd5_corpus shape",sum_weighted_projection_vector_svd5_corpus.shape)
#             print("\n sum_weighted_projection_vector_svd3_citant shape",sum_weighted_projection_vector_svd3_citant.shape)
#             print("\n sum_weighted_projection_vector_svd5_citant shape",sum_weighted_projection_vector_svd5_citant.shape)
   
            
            
            ## finally removing the common components from the embeddings: 
            
            

            
            corpus_pca_removed3_embed = corpus_sentence_embed_arr - sum_weighted_projection_vector_svd3_corpus
            corpus_pca_removed5_embed = corpus_sentence_embed_arr - sum_weighted_projection_vector_svd5_corpus
            citant_pca_removed3_embed = citance_sentence_embed_arr - sum_weighted_projection_vector_svd3_citant
            citant_pca_removed5_embed = citance_sentence_embed_arr - sum_weighted_projection_vector_svd5_citant
            
            
#             print("\n corpus_pca_removed3_embed shape",corpus_pca_removed3_embed.shape)
#             print("\n corpus_pca_removed5_embed shape",corpus_pca_removed5_embed.shape)
#             print("\n citant_pca_removed3_embed shape",citant_pca_removed3_embed.shape)
#             print("\n citant_pca_removed5_embed shape",citant_pca_removed5_embed.shape)
            
            dictionary[keys]['corpus_pca_removed3_embed'] = corpus_pca_removed3_embed
            dictionary[keys]['corpus_pca_removed5_embed'] = corpus_pca_removed5_embed
            dictionary[keys]['citant_pca_removed3_embed'] = citant_pca_removed3_embed
            dictionary[keys]['citant_pca_removed5_embed'] = citant_pca_removed5_embed
   
    with open(des, 'wb') as handle:
        pickle.dump(dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        
        
        
#         corpus_pca_removed3_embed = 
#         corpus_pca_removed5_embed = 
        
#         citant_pca_removed3_embed =  
#         citant_pca_removed5_embed = 
        
        
        
        
        
     

## removing pca from test docs 2018 

In [7]:
with open('test_docs.pkl', 'rb') as f:
    dic_Test_2018 = pickle.load(f)

create_weightedPCAremoved(dic_Test_2018, des = 'updated_test_2018.pkl')


 corpus_sentence_embed_arr shape  (288, 768)

 citance_sentence_embed_arr shape (20, 768)

 corpus_sentence_embed_arr shape  (147, 768)

 citance_sentence_embed_arr shape (15, 768)

 corpus_sentence_embed_arr shape  (415, 768)

 citance_sentence_embed_arr shape (17, 768)

 corpus_sentence_embed_arr shape  (257, 768)

 citance_sentence_embed_arr shape (13, 768)

 corpus_sentence_embed_arr shape  (179, 768)

 citance_sentence_embed_arr shape (18, 768)

 corpus_sentence_embed_arr shape  (203, 768)

 citance_sentence_embed_arr shape (17, 768)

 corpus_sentence_embed_arr shape  (176, 768)

 citance_sentence_embed_arr shape (16, 768)

 corpus_sentence_embed_arr shape  (198, 768)

 citance_sentence_embed_arr shape (16, 768)

 corpus_sentence_embed_arr shape  (113, 768)

 citance_sentence_embed_arr shape (20, 768)

 corpus_sentence_embed_arr shape  (154, 768)

 citance_sentence_embed_arr shape (19, 768)

 corpus_sentence_embed_arr shape  (194, 768)

 citance_sentence_embed_arr shape (17, 768)

In [12]:
updated_dic_Test_2018['W11-2123']['citant_pca_removed5_embed'].shape

(20, 768)

In [13]:
with open('updated_test_2018.pkl', 'rb') as f:
    updated_dic_Test_2018 = pickle.load(f)

In [14]:
len(updated_dic_Test_2018)

20

## pca removal for 2018 train

In [14]:
with open('train_docs.pkl', 'rb') as f:
    dic_Train_2018 = pickle.load(f)

create_weightedPCAremoved(dic_Train_2018, des = 'updated_train_2018.pkl')


 count : 1

 corpus_sentence_embed_arr shape  (214, 768)

 citance_sentence_embed_arr shape (26, 768)

 count : 2

 corpus_sentence_embed_arr shape  (100, 768)

 citance_sentence_embed_arr shape (12, 768)

 count : 3

 corpus_sentence_embed_arr shape  (156, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 4

 corpus_sentence_embed_arr shape  (205, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 5

 corpus_sentence_embed_arr shape  (227, 768)

 citance_sentence_embed_arr shape (10, 768)

 count : 6

 corpus_sentence_embed_arr shape  (338, 768)

 citance_sentence_embed_arr shape (37, 768)

 count : 7

 corpus_sentence_embed_arr shape  (165, 768)

 citance_sentence_embed_arr shape (8, 768)

 count : 8

 corpus_sentence_embed_arr shape  (247, 768)

 citance_sentence_embed_arr shape (15, 768)

 count : 9

 corpus_sentence_embed_arr shape  (177, 768)

 citance_sentence_embed_arr shape (15, 768)

 count : 10

 corpus_sentence_embed_arr shape  (214, 768)

 citance_sente

In [16]:
with open('updated_train_2018.pkl', 'rb') as f:
    updated_dic_Train_2018 = pickle.load(f)
    
len(updated_dic_Train_2018.keys())    

29

In [20]:
updated_dic_Train_2018['P07-1040']['citant_pca_removed5_embed'].shape

(26, 768)

## pca removal for 2019 train

In [21]:
with open('train_docs_2019.pkl', 'rb') as f:
    dic_Train_2019 = pickle.load(f)

create_weightedPCAremoved(dic_Train_2019, des = 'updated_train_2019.pkl')


 count : 1

 corpus_sentence_embed_arr shape  (179, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 2

 corpus_sentence_embed_arr shape  (202, 768)

 citance_sentence_embed_arr shape (17, 768)

 count : 3

 corpus_sentence_embed_arr shape  (148, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 4

 corpus_sentence_embed_arr shape  (157, 768)

 citance_sentence_embed_arr shape (14, 768)

 count : 5

 corpus_sentence_embed_arr shape  (192, 768)

 citance_sentence_embed_arr shape (17, 768)

 count : 6

 corpus_sentence_embed_arr shape  (178, 768)

 citance_sentence_embed_arr shape (13, 768)

 count : 7

 corpus_sentence_embed_arr shape  (129, 768)

 citance_sentence_embed_arr shape (10, 768)

 count : 8

 corpus_sentence_embed_arr shape  (129, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 9

 corpus_sentence_embed_arr shape  (223, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 10

 corpus_sentence_embed_arr shape  (126, 768)

 citance_sent

/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered


 count : 31

 corpus_sentence_embed_arr shape  (161, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 32

 corpus_sentence_embed_arr shape  (1, 768)

 citance_sentence_embed_arr shape (17, 768)

 count : 33

 corpus_sentence_embed_arr shape  (214, 768)

 citance_sentence_embed_arr shape (10, 768)

 count : 34

 corpus_sentence_embed_arr shape  (23, 768)

 citance_sentence_embed_arr shape (17, 768)

 count : 35

 corpus_sentence_embed_arr shape  (203, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 36

 corpus_sentence_embed_arr shape  (1, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 37

 corpus_sentence_embed_arr shape  (149, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 38

 corpus_sentence_embed_arr shape  (33, 768)

 citance_sentence_embed_arr shape (18, 768)


/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered


 count : 39

 corpus_sentence_embed_arr shape  (188, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 40

 corpus_sentence_embed_arr shape  (1, 768)

 citance_sentence_embed_arr shape (16, 768)

 count : 41

 corpus_sentence_embed_arr shape  (181, 768)

 citance_sentence_embed_arr shape (15, 768)

 count : 42

 corpus_sentence_embed_arr shape  (178, 768)

 citance_sentence_embed_arr shape (10, 768)

 count : 43

 corpus_sentence_embed_arr shape  (211, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 44

 corpus_sentence_embed_arr shape  (1, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 45

 corpus_sentence_embed_arr shape  (120, 768)

 citance_sentence_embed_arr shape (20, 768)


...

/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var



 count : 363

 corpus_sentence_embed_arr shape  (214, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 364

 corpus_sentence_embed_arr shape  (216, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 365

 corpus_sentence_embed_arr shape  (211, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 366

 corpus_sentence_embed_arr shape  (163, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 367

 corpus_sentence_embed_arr shape  (216, 768)

 citance_sentence_embed_arr shape (12, 768)

 count : 368

 corpus_sentence_embed_arr shape  (212, 768)

 citance_sentence_embed_arr shape (15, 768)

 count : 369

 corpus_sentence_embed_arr shape  (253, 768)

 citance_sentence_embed_arr shape (14, 768)

 count : 370

 corpus_sentence_embed_arr shape  (282, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 371

 corpus_sentence_embed_arr shape  (208, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 372

 corpus_sentence_embed_arr shape  (185, 


 count : 446

 corpus_sentence_embed_arr shape  (132, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 447

 corpus_sentence_embed_arr shape  (173, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 448

 corpus_sentence_embed_arr shape  (237, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 449

 corpus_sentence_embed_arr shape  (147, 768)

 citance_sentence_embed_arr shape (17, 768)

 count : 450

 corpus_sentence_embed_arr shape  (107, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 451

 corpus_sentence_embed_arr shape  (25, 768)

 citance_sentence_embed_arr shape (17, 768)

 count : 452

 corpus_sentence_embed_arr shape  (155, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 453

 corpus_sentence_embed_arr shape  (124, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 454

 corpus_sentence_embed_arr shape  (211, 768)

 citance_sentence_embed_arr shape (12, 768)

 count : 455

 corpus_sentence_embed_arr shape  (177, 7


 count : 531

 corpus_sentence_embed_arr shape  (171, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 532

 corpus_sentence_embed_arr shape  (134, 768)

 citance_sentence_embed_arr shape (17, 768)

 count : 533

 corpus_sentence_embed_arr shape  (206, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 534

 corpus_sentence_embed_arr shape  (132, 768)

 citance_sentence_embed_arr shape (11, 768)

 count : 535

 corpus_sentence_embed_arr shape  (105, 768)

 citance_sentence_embed_arr shape (10, 768)

 count : 536

 corpus_sentence_embed_arr shape  (205, 768)

 citance_sentence_embed_arr shape (16, 768)

 count : 537

 corpus_sentence_embed_arr shape  (215, 768)

 citance_sentence_embed_arr shape (15, 768)

 count : 538

 corpus_sentence_embed_arr shape  (194, 768)

 citance_sentence_embed_arr shape (10, 768)

 count : 539

 corpus_sentence_embed_arr shape  (160, 768)

 citance_sentence_embed_arr shape (8, 768)

 count : 540

 corpus_sentence_embed_arr shape  (173, 7


 count : 615

 corpus_sentence_embed_arr shape  (203, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 616

 corpus_sentence_embed_arr shape  (143, 768)

 citance_sentence_embed_arr shape (10, 768)

 count : 617

 corpus_sentence_embed_arr shape  (132, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 618

 corpus_sentence_embed_arr shape  (230, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 619

 corpus_sentence_embed_arr shape  (181, 768)

 citance_sentence_embed_arr shape (17, 768)

 count : 620

 corpus_sentence_embed_arr shape  (161, 768)

 citance_sentence_embed_arr shape (15, 768)

 count : 621

 corpus_sentence_embed_arr shape  (152, 768)

 citance_sentence_embed_arr shape (17, 768)

 count : 622

 corpus_sentence_embed_arr shape  (150, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 623

 corpus_sentence_embed_arr shape  (160, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 624

 corpus_sentence_embed_arr shape  (194, 

In [2]:
import pickle
with open('updated_train_2019.pkl', 'rb') as f:
    updated_dic_Train_2019 = pickle.load(f)
    
print(len(updated_dic_Train_2019.keys()))    
updated_dic_Train_2019.keys()#['P07-1040']['citant_pca_removed5_embed'].shape

651


dict_keys(['A00-1031', 'A00-1043', 'A00-2004', 'A00-2009', 'A00-2019', 'A00-2024', 'A00-2026', 'A00-2031', 'A00-2034', 'A88-1019', 'A92-1006', 'A92-1018', 'A92-1021', 'A94-1006', 'A94-1009', 'A94-1016', 'A97-1004', 'A97-1011', 'A97-1029', 'A97-1030', 'A97-1039', 'A97-1052', 'C00-1007', 'C00-1044', 'C00-1072', 'C00-2136', 'C00-2137', 'C00-2163', 'C02-1011', 'C02-1054', 'C02-1114', 'C02-1139', 'C02-1144', 'C02-1145', 'C02-1150', 'C02-2025', 'C04-1010', 'C04-1024', 'C04-1041', 'C04-1046', 'C04-1051', 'C04-1059', 'C04-1072', 'C04-1073', 'C04-1080', 'C04-1081', 'C04-1100', 'C04-1111', 'C04-1146', 'C04-1180', 'C04-1197', 'C04-1200', 'C08-1018', 'C08-1022', 'C08-1107', 'C08-1109', 'C08-1114', 'C10-1011', 'C10-1152', 'C10-2005', 'C10-2028', 'C86-1016', 'C86-1045', 'C88-1016', 'C88-2121', 'C88-2128', 'C88-2147', 'C90-2067', 'C90-3030', 'C90-3044', 'C90-3045', 'C90-3052', 'C90-3063', 'C92-1019', 'C92-1025', 'C92-1038', 'C92-2066', 'C92-2070', 'C92-2082', 'C92-3126', 'C92-3150', 'C94-1027', 'C94-

In [4]:
import sys
sys.getsizeof(updated_dic_Train_2019)

18520

## pca removal for 2019 test

In [7]:
with open('test_docs_2019.pkl', 'rb') as f:
    dic_Test_2019 = pickle.load(f)

create_weightedPCAremoved(dic_Test_2019, des = 'updated_test_2019.pkl')


 count : 1

 corpus_sentence_embed_arr shape  (169, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 2

 corpus_sentence_embed_arr shape  (227, 768)

 citance_sentence_embed_arr shape (14, 768)

 count : 3

 corpus_sentence_embed_arr shape  (86, 768)

 citance_sentence_embed_arr shape (15, 768)

 count : 4

 corpus_sentence_embed_arr shape  (84, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 5

 corpus_sentence_embed_arr shape  (105, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 6

 corpus_sentence_embed_arr shape  (208, 768)

 citance_sentence_embed_arr shape (7, 768)

 count : 7

 corpus_sentence_embed_arr shape  (198, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 8

 corpus_sentence_embed_arr shape  (226, 768)

 citance_sentence_embed_arr shape (12, 768)

 count : 9

 corpus_sentence_embed_arr shape  (215, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 10

 corpus_sentence_embed_arr shape  (178, 768)

 citance_sentenc

/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var



 count : 36

 corpus_sentence_embed_arr shape  (208, 768)

 citance_sentence_embed_arr shape (5, 768)

 count : 37

 corpus_sentence_embed_arr shape  (1, 768)

 citance_sentence_embed_arr shape (16, 768)

 count : 38

 corpus_sentence_embed_arr shape  (166, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 39

 corpus_sentence_embed_arr shape  (245, 768)

 citance_sentence_embed_arr shape (3, 768)

 count : 40

 corpus_sentence_embed_arr shape  (143, 768)

 citance_sentence_embed_arr shape (3, 768)

 count : 41

 corpus_sentence_embed_arr shape  (1, 768)

 citance_sentence_embed_arr shape (10, 768)

 count : 42

 corpus_sentence_embed_arr shape  (143, 768)

 citance_sentence_embed_arr shape (3, 768)

 count : 43

 corpus_sentence_embed_arr shape  (126, 768)

 citance_sentence_embed_arr shape (16, 768)

 count : 44

 corpus_sentence_embed_arr shape  (201, 768)

 citance_sentence_embed_arr shape (3, 768)

 count : 45

 corpus_sentence_embed_arr shape  (146, 768)

 citance_sent

/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var



 count : 46

 corpus_sentence_embed_arr shape  (180, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 47

 corpus_sentence_embed_arr shape  (126, 768)

 citance_sentence_embed_arr shape (4, 768)

 count : 48

 corpus_sentence_embed_arr shape  (159, 768)

 citance_sentence_embed_arr shape (4, 768)

 count : 49

 corpus_sentence_embed_arr shape  (93, 768)

 citance_sentence_embed_arr shape (14, 768)

 count : 50

 corpus_sentence_embed_arr shape  (175, 768)

 citance_sentence_embed_arr shape (5, 768)

 count : 51

 corpus_sentence_embed_arr shape  (186, 768)

 citance_sentence_embed_arr shape (8, 768)

 count : 52

 corpus_sentence_embed_arr shape  (138, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 53

 corpus_sentence_embed_arr shape  (184, 768)

 citance_sentence_embed_arr shape (7, 768)

 count : 54

 corpus_sentence_embed_arr shape  (151, 768)

 citance_sentence_embed_arr shape (14, 768)

 count : 55

 corpus_sentence_embed_arr shape  (193, 768)

 citance_s


 count : 133

 corpus_sentence_embed_arr shape  (62, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 134

 corpus_sentence_embed_arr shape  (74, 768)

 citance_sentence_embed_arr shape (12, 768)

 count : 135

 corpus_sentence_embed_arr shape  (169, 768)

 citance_sentence_embed_arr shape (13, 768)

 count : 136

 corpus_sentence_embed_arr shape  (153, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 137

 corpus_sentence_embed_arr shape  (148, 768)

 citance_sentence_embed_arr shape (13, 768)

 count : 138

 corpus_sentence_embed_arr shape  (166, 768)

 citance_sentence_embed_arr shape (17, 768)

 count : 139

 corpus_sentence_embed_arr shape  (137, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 140

 corpus_sentence_embed_arr shape  (175, 768)

 citance_sentence_embed_arr shape (15, 768)

 count : 141

 corpus_sentence_embed_arr shape  (202, 768)

 citance_sentence_embed_arr shape (12, 768)

 count : 142

 corpus_sentence_embed_arr shape  (227, 76


 count : 213

 corpus_sentence_embed_arr shape  (152, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 214

 corpus_sentence_embed_arr shape  (111, 768)

 citance_sentence_embed_arr shape (13, 768)

 count : 215

 corpus_sentence_embed_arr shape  (182, 768)

 citance_sentence_embed_arr shape (15, 768)

 count : 216

 corpus_sentence_embed_arr shape  (146, 768)

 citance_sentence_embed_arr shape (15, 768)

 count : 217

 corpus_sentence_embed_arr shape  (198, 768)

 citance_sentence_embed_arr shape (19, 768)


In [1]:
import pickle
with open('updated_test_2019.pkl', 'rb') as f:
    updated_dic_Test_2019 = pickle.load(f)
    
print(len(updated_dic_Test_2019.keys()))    
updated_dic_Test_2019.keys()#['P07-1040']['citant_pca_removed5_embed'].shape

217


dict_keys(['P10-1110', 'P10-1142', 'P10-1146', 'P10-2041', 'P10-4002', 'P11-1016', 'P11-1019', 'P11-1020', 'P11-1038', 'P11-1055', 'P11-1098', 'P11-1138', 'P11-2008', 'P11-2031', 'P11-2033', 'P12-1092', 'P13-1045', 'P83-1007', 'P83-1019', 'P83-1020', 'P83-1021', 'P84-1008', 'P84-1018', 'P84-1075', 'P84-1085', 'P85-1008', 'P85-1011', 'P85-1018', 'P86-1004', 'P86-1031', 'P87-1022', 'P87-1033', 'P88-1012', 'P88-1015', 'P88-1020', 'P89-1002', 'P89-1009', 'P89-1010', 'P89-1031', 'P90-1005', 'P90-1010', 'P90-1032', 'P90-1034', 'P91-1017', 'P91-1022', 'P91-1023', 'P91-1027', 'P91-1030', 'P91-1034', 'P92-1005', 'P92-1008', 'P92-1017', 'P92-1032', 'P93-1001', 'P93-1002', 'P93-1003', 'P93-1005', 'P93-1008', 'P93-1016', 'P93-1020', 'P93-1022', 'P93-1023', 'P93-1024', 'P93-1032', 'P93-1035', 'P93-1041', 'P94-1002', 'P94-1012', 'P94-1013', 'P94-1019', 'P94-1020', 'P95-1007', 'P95-1021', 'P95-1026', 'P95-1034', 'P95-1037', 'P95-1050', 'P96-1006', 'P96-1008', 'P96-1011', 'P96-1021', 'P96-1024', 'P96-

In [3]:
updated_dic_Test_2019['P10-1142']['citant_pca_removed5_embed'].shape

(14, 768)

In [5]:
updated_dic_Test_2019['P10-1142']['corpus_pca_removed5_embed'].shape

(227, 768)

In [1]:
!ls


2019
av_modular_2018_19.ipynb
av_modular_2018_19_ngrams.ipynb
av_modular_2019_wmd_final_results.ipynb
backup
bert2018_19_full_dset_ngrams.pth
bert2018_19_full_dset.pth
bert2018_fine_tuned.pth
bert2018_full_dset_ngrams.pth
bert2018_full_dset.pth
bert2019_full_dset_ngrams.pth
bert2019_full_dset.pth
just_rough_work_to_test.ipynb
main_full_dset_trained_normal_test_ngram.ipynb
main_full_dset_train_test_normal.ipynb
main_full_dset_train_test_normal_wmd-Copy1.ipynb
main_full_dset_train_test_normal_wmd.ipynb
main_full_dset_train_test_normal_wmd_top5.ipynb
modular_2018_wmd-Copy1.ipynb
modular_2018_wmd.ipynb
modular_2019_ngram.ipynb
modular_2019_wmd_-Copy1.ipynb
modular_2019_wmd-Copy6_ngram.ipynb
modular_2019_wmd_.ipynb
modular_2019_wmd.ipynb
modules
ngrams2018.py
__pycache__
results_all_2018_top10.pkl
results_all_2018_top3.pkl
results_all_2018_top5.pkl
results_all_2019_normal.pkl
results_all_2019.pkl
results_all_2019_top10.pkl
results_all_2019_top3.pkl
results_all_2019_top5.pkl
run_ngrams.py
ru

In [ ]:
test_docs_2018_19.pkl

## PCA removal of 2018-19 normal (non ngram) version.

In [4]:
with open('test_docs_2018_19.pkl', 'rb') as f:
    dic_test_docs_2018_19 = pickle.load(f)

create_weightedPCAremoved(dic_test_docs_2018_19, des = 'updated_test_docs_2018_19.pkl')


 count : 1

 corpus_sentence_embed_arr shape  (179, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 2

 corpus_sentence_embed_arr shape  (149, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 3

 corpus_sentence_embed_arr shape  (198, 768)

 citance_sentence_embed_arr shape (16, 768)

 count : 4

 corpus_sentence_embed_arr shape  (257, 768)

 citance_sentence_embed_arr shape (13, 768)

 count : 5

 corpus_sentence_embed_arr shape  (142, 768)

 citance_sentence_embed_arr shape (16, 768)

 count : 6

 corpus_sentence_embed_arr shape  (200, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 7

 corpus_sentence_embed_arr shape  (203, 768)

 citance_sentence_embed_arr shape (17, 768)

 count : 8

 corpus_sentence_embed_arr shape  (175, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 9

 corpus_sentence_embed_arr shape  (194, 768)

 citance_sentence_embed_arr shape (17, 768)

 count : 10

 corpus_sentence_embed_arr shape  (111, 768)

 citance_sent

/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var



 count : 51

 corpus_sentence_embed_arr shape  (111, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 52

 corpus_sentence_embed_arr shape  (143, 768)

 citance_sentence_embed_arr shape (8, 768)

 count : 53

 corpus_sentence_embed_arr shape  (267, 768)

 citance_sentence_embed_arr shape (5, 768)

 count : 54

 corpus_sentence_embed_arr shape  (146, 768)

 citance_sentence_embed_arr shape (13, 768)

 count : 55

 corpus_sentence_embed_arr shape  (167, 768)

 citance_sentence_embed_arr shape (3, 768)

 count : 56

 corpus_sentence_embed_arr shape  (208, 768)

 citance_sentence_embed_arr shape (5, 768)

 count : 57

 corpus_sentence_embed_arr shape  (1, 768)

 citance_sentence_embed_arr shape (16, 768)

 count : 58

 corpus_sentence_embed_arr shape  (166, 768)

 citance_sentence_embed_arr shape (19, 768)


/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home2/dhawals1939/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var



 count : 59

 corpus_sentence_embed_arr shape  (245, 768)

 citance_sentence_embed_arr shape (3, 768)

 count : 60

 corpus_sentence_embed_arr shape  (143, 768)

 citance_sentence_embed_arr shape (3, 768)

 count : 61

 corpus_sentence_embed_arr shape  (1, 768)

 citance_sentence_embed_arr shape (10, 768)

 count : 62

 corpus_sentence_embed_arr shape  (143, 768)

 citance_sentence_embed_arr shape (3, 768)

 count : 63

 corpus_sentence_embed_arr shape  (126, 768)

 citance_sentence_embed_arr shape (16, 768)

 count : 64

 corpus_sentence_embed_arr shape  (201, 768)

 citance_sentence_embed_arr shape (3, 768)

 count : 65

 corpus_sentence_embed_arr shape  (146, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 66

 corpus_sentence_embed_arr shape  (180, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 67

 corpus_sentence_embed_arr shape  (126, 768)

 citance_sentence_embed_arr shape (4, 768)

 count : 68

 corpus_sentence_embed_arr shape  (159, 768)

 citance_se


 count : 138

 corpus_sentence_embed_arr shape  (172, 768)

 citance_sentence_embed_arr shape (7, 768)

 count : 139

 corpus_sentence_embed_arr shape  (147, 768)

 citance_sentence_embed_arr shape (15, 768)

 count : 140

 corpus_sentence_embed_arr shape  (139, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 141

 corpus_sentence_embed_arr shape  (121, 768)

 citance_sentence_embed_arr shape (14, 768)

 count : 142

 corpus_sentence_embed_arr shape  (166, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 143

 corpus_sentence_embed_arr shape  (131, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 144

 corpus_sentence_embed_arr shape  (141, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 145

 corpus_sentence_embed_arr shape  (185, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 146

 corpus_sentence_embed_arr shape  (96, 768)

 citance_sentence_embed_arr shape (11, 768)

 count : 147

 corpus_sentence_embed_arr shape  (131, 76


 count : 220

 corpus_sentence_embed_arr shape  (167, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 221

 corpus_sentence_embed_arr shape  (182, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 222

 corpus_sentence_embed_arr shape  (163, 768)

 citance_sentence_embed_arr shape (19, 768)

 count : 223

 corpus_sentence_embed_arr shape  (144, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 224

 corpus_sentence_embed_arr shape  (167, 768)

 citance_sentence_embed_arr shape (17, 768)

 count : 225

 corpus_sentence_embed_arr shape  (205, 768)

 citance_sentence_embed_arr shape (20, 768)

 count : 226

 corpus_sentence_embed_arr shape  (200, 768)

 citance_sentence_embed_arr shape (18, 768)

 count : 227

 corpus_sentence_embed_arr shape  (209, 768)

 citance_sentence_embed_arr shape (11, 768)

 count : 228

 corpus_sentence_embed_arr shape  (158, 768)

 citance_sentence_embed_arr shape (12, 768)

 count : 229

 corpus_sentence_embed_arr shape  (86, 7

In [5]:
with open('updated_test_docs_2018_19.pkl', 'rb') as f:
    updated_test_docs_2018_19 = pickle.load(f)
    
print(len(updated_test_docs_2018_19.keys()))    
updated_test_docs_2018_19.keys()#['P07-1040']['citant_pca_removed5_embed'].shape

237


dict_keys(['W06-3114', 'W99-0623', 'P08-1043', 'W99-0613', 'P08-1102', 'D09-1092', 'P08-1028', 'P11-1060', 'P04-1036', 'W06-2932', 'P05-1013', 'D10-1044', 'A97-1014', 'P87-1015', 'W11-2123', 'A00-2018', 'E03-1005', 'P11-1061', 'A00-2030', 'J01-2004', 'P10-1110', 'P10-1142', 'P10-1146', 'P10-2041', 'P10-4002', 'P11-1016', 'P11-1019', 'P11-1020', 'P11-1038', 'P11-1055', 'P11-1098', 'P11-1138', 'P11-2008', 'P11-2031', 'P11-2033', 'P12-1092', 'P13-1045', 'P83-1007', 'P83-1019', 'P83-1020', 'P83-1021', 'P84-1008', 'P84-1018', 'P84-1075', 'P84-1085', 'P85-1008', 'P85-1011', 'P85-1018', 'P86-1004', 'P86-1031', 'P87-1022', 'P87-1033', 'P88-1012', 'P88-1015', 'P88-1020', 'P89-1002', 'P89-1009', 'P89-1010', 'P89-1031', 'P90-1005', 'P90-1010', 'P90-1032', 'P90-1034', 'P91-1017', 'P91-1022', 'P91-1023', 'P91-1027', 'P91-1030', 'P91-1034', 'P92-1005', 'P92-1008', 'P92-1017', 'P92-1032', 'P93-1001', 'P93-1002', 'P93-1003', 'P93-1005', 'P93-1008', 'P93-1016', 'P93-1020', 'P93-1022', 'P93-1023', 'P93-

In [8]:
updated_test_docs_2018_19['W06-3114']['citant_pca_removed5_embed'].shape

(18, 768)

## pca removal for ngrams

In [1]:
## the file path

import pickle, sys
import numpy as np
from scipy.spatial import distance
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.decomposition import TruncatedSVD
import gensim.models.keyedvectors as word2vec


    

    
def create_weightedPCAremoved_ngram(dictionary, des):
    
    
    corpus_pca_removed1_embed = []
    corpus_pca_removed3_embed = []
    corpus_pca_removed5_embed = []
    citant_pca_removed1_embed = []
    citant_pca_removed3_embed = []
    citant_pca_removed5_embed = []
    
    count = 0
    
    for keys in dictionary.keys():
#         if count < 2:
            count+=1
            
            ## embedding vectors of corpus and citant sentences
            
            original_corpus_shape = np.array(dictionary[keys]['corpus_ngram_embed']).shape
            original_citant_shape = np.array(dictionary[keys]['citant_ngram_embed']).shape
        
            corpus_sentence_embed_arr = np.array(dictionary[keys]['corpus_ngram_embed']).reshape(-1,768)
            citance_sentence_embed_arr = np.array(dictionary[keys]['citant_ngram_embed']).reshape(-1,768) # 18 x 768
            print("\n count :", count)
            print("\n corpus_sentence_embed_arr shape ", corpus_sentence_embed_arr.shape)
            print("\n citance_sentence_embed_arr shape", citance_sentence_embed_arr.shape)

            svd3_corpus = TruncatedSVD(n_components=3, random_state=0).fit(corpus_sentence_embed_arr)
            svd3_citant = TruncatedSVD(n_components=3, random_state=0).fit(citance_sentence_embed_arr)

            svd5_corpus = TruncatedSVD(n_components=5, random_state=0).fit(corpus_sentence_embed_arr)
            svd5_citant = TruncatedSVD(n_components=5, random_state=0).fit(citance_sentence_embed_arr)

            ## normalised eigen values to find weighted projections on eigen vectors/ principal directions
            svd3_corpus_singular_values = svd3_corpus.singular_values_/svd3_corpus.singular_values_.sum()
            svd5_corpus_singular_values = svd5_corpus.singular_values_/svd5_corpus.singular_values_.sum()

            svd3_citant_singular_values = svd3_citant.singular_values_/svd3_citant.singular_values_.sum()
            svd5_citant_singular_values = svd5_citant.singular_values_/svd5_citant.singular_values_.sum()

            
            ## principal directions for corpus and citant sentences
            svd3_corpus_comp = svd3_corpus.components_ # 3 x 768
            svd5_corpus_comp = svd5_corpus.components_ # 5 x 768

            svd3_citant_comp = svd3_citant.components_
            svd5_citant_comp = svd5_citant.components_

#             print("\n svd3_corpus_comp shape",svd3_corpus_comp.shape)
#             print("\n svd5_corpus_comp shape",svd5_corpus_comp.shape)
#             print("\n svd3_citant_comp shape",svd3_citant_comp.shape)
#             print("\n svd5_citant_comp shape",svd5_citant_comp.shape)

            
            
            ## scaler value of embeddings projected on the principal directions. We will multiply
            ## them with the eigen values and Then we will multiply
            ## it with unit eigen vector/ principal directions to find the common component vectors.
            ## then we will add them all. and subtract from the embeddings.
            
            
            ##scaler values
            svd3_corpus_projections = corpus_sentence_embed_arr@svd3_corpus_comp.T #(203, 3) number of sen x no of pc
            svd5_corpus_projections = corpus_sentence_embed_arr@svd5_corpus_comp.T #(203, 5)

            svd3_citant_projections = citance_sentence_embed_arr@svd3_citant_comp.T
            svd5_citant_projections = citance_sentence_embed_arr@svd5_citant_comp.T

#             print("\n svd3_corpus_projections shape",svd3_corpus_projections.shape)
#             print("\n svd5_corpus_projections shape",svd5_corpus_projections.shape)
#             print("\n svd3_citant_projections shape",svd3_citant_projections.shape)
#             print("\n svd5_citant_projections shape",svd5_citant_projections.shape)

            
            ##sclaer values of projections after multiplying with the eigen values (point wise)
            value_weighted_svd3_corpus_projections = svd3_corpus_singular_values * svd3_corpus_projections
            value_weighted_svd5_corpus_projections = svd5_corpus_singular_values * svd5_corpus_projections
            value_weighted_svd3_citant_projections = svd3_citant_singular_values * svd3_citant_projections
            value_weighted_svd5_citant_projections = svd5_citant_singular_values * svd5_citant_projections
            
#             print("\n value_weighted_svd3_corpus_projections shape",value_weighted_svd3_corpus_projections.shape)
#             print("\n value_weighted_svd5_corpus_projections shape",value_weighted_svd5_corpus_projections.shape)
#             print("\n value_weighted_svd3_citant_projections shape",value_weighted_svd3_citant_projections.shape)
#             print("\n value_weighted_svd5_citant_projections shape",value_weighted_svd5_citant_projections.shape)
   
            
    
            ##reshaping
            new_value_weighted_svd3_corpus_projections = value_weighted_svd3_corpus_projections[:,None,:]
            new_value_weighted_svd5_corpus_projections = value_weighted_svd5_corpus_projections[:,None,:]
            new_value_weighted_svd3_citant_projections = value_weighted_svd3_citant_projections[:,None,:]
            new_value_weighted_svd5_citant_projections = value_weighted_svd5_citant_projections[:,None,:]
            
            
            ##reshaping
            new_svd3_corpus_comp = svd3_corpus_comp.T[None,:]
            new_svd5_corpus_comp = svd5_corpus_comp.T[None,:]
            new_svd3_citant_comp = svd3_citant_comp.T[None,:]
            new_svd5_citant_comp = svd5_citant_comp.T[None,:]
            
#             print("\n new_svd3_corpus_comp shape",new_svd3_corpus_comp.shape)
#             print("\n new_svd5_corpus_comp shape",new_svd5_corpus_comp.shape)
#             print("\n new_svd3_citant_comp shape",new_svd3_citant_comp.shape)
#             print("\n new_svd5_citant_comp shape",new_svd5_citant_comp.shape)
   
            
            
            ## multiplying to get vectors
            weighted_projection_vector_svd3_corpus = new_value_weighted_svd3_corpus_projections*new_svd3_corpus_comp
            weighted_projection_vector_svd5_corpus = new_value_weighted_svd5_corpus_projections*new_svd5_corpus_comp
            weighted_projection_vector_svd3_citant = new_value_weighted_svd3_citant_projections*new_svd3_citant_comp
            weighted_projection_vector_svd5_citant = new_value_weighted_svd5_citant_projections*new_svd5_citant_comp
            
            
            sum_weighted_projection_vector_svd3_corpus = weighted_projection_vector_svd3_corpus.sum(axis=2)
            sum_weighted_projection_vector_svd5_corpus = weighted_projection_vector_svd5_corpus.sum(axis=2)
            sum_weighted_projection_vector_svd3_citant = weighted_projection_vector_svd3_citant.sum(axis=2)
            sum_weighted_projection_vector_svd5_citant = weighted_projection_vector_svd5_citant.sum(axis=2)
            
#             print("\n sum_weighted_projection_vector_svd3_corpus shape",sum_weighted_projection_vector_svd3_corpus.shape)
#             print("\n sum_weighted_projection_vector_svd5_corpus shape",sum_weighted_projection_vector_svd5_corpus.shape)
#             print("\n sum_weighted_projection_vector_svd3_citant shape",sum_weighted_projection_vector_svd3_citant.shape)
#             print("\n sum_weighted_projection_vector_svd5_citant shape",sum_weighted_projection_vector_svd5_citant.shape)
   
            
            
            ## finally removing the common components from the embeddings: 
            
            

            
            corpus_pca_removed3_embed = corpus_sentence_embed_arr - sum_weighted_projection_vector_svd3_corpus
            corpus_pca_removed5_embed = corpus_sentence_embed_arr - sum_weighted_projection_vector_svd5_corpus
            citant_pca_removed3_embed = citance_sentence_embed_arr - sum_weighted_projection_vector_svd3_citant
            citant_pca_removed5_embed = citance_sentence_embed_arr - sum_weighted_projection_vector_svd5_citant
            
            
#             print("\n corpus_pca_removed3_embed shape",corpus_pca_removed3_embed.shape)
#             print("\n corpus_pca_removed5_embed shape",corpus_pca_removed5_embed.shape)
#             print("\n citant_pca_removed3_embed shape",citant_pca_removed3_embed.shape)
#             print("\n citant_pca_removed5_embed shape",citant_pca_removed5_embed.shape)
            
            dictionary[keys]['corpus_ngram_pca_removed3_embed'] = corpus_pca_removed3_embed.reshape(original_corpus_shape)
            dictionary[keys]['corpus_ngram_pca_removed5_embed'] = corpus_pca_removed5_embed.reshape(original_corpus_shape)
            dictionary[keys]['citant_ngram_pca_removed3_embed'] = citant_pca_removed3_embed.reshape(original_citant_shape)
            dictionary[keys]['citant_ngram_pca_removed5_embed'] = citant_pca_removed5_embed.reshape(original_citant_shape)
   
    with open(des, 'wb') as handle:
        pickle.dump(dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
        
        
        
#         corpus_pca_removed3_embed = 
#         corpus_pca_removed5_embed = 
        
#         citant_pca_removed3_embed =  
#         citant_pca_removed5_embed = 
        
        
        
        
        
     

In [40]:
import pickle5 as pickle
path = '/ssd_scratch/cvit/dhawals1939/'
for k in [0,15,19]:
    file = path+'test_docs_2018_ngrams_num_'+str(k)+'.pkl'
    with open(file,'rb') as f:
        d = pickle.load(f)
    new_file = file.replace('test','updated_test')
    print(new_file)
    create_weightedPCAremoved_ngram(d, des = new_file)


/ssd_scratch/cvit/dhawals1939/updated_test_docs_2018_ngrams_num_0.pkl

 count : 1

 corpus_sentence_embed_arr shape  (3728, 768)

 citance_sentence_embed_arr shape (3728, 768)
/ssd_scratch/cvit/dhawals1939/updated_test_docs_2018_ngrams_num_15.pkl

 count : 1

 corpus_sentence_embed_arr shape  (2205, 768)

 citance_sentence_embed_arr shape (2205, 768)

 count : 2

 corpus_sentence_embed_arr shape  (7055, 768)

 citance_sentence_embed_arr shape (7055, 768)

 count : 3

 corpus_sentence_embed_arr shape  (3341, 768)

 citance_sentence_embed_arr shape (3341, 768)

 count : 4

 corpus_sentence_embed_arr shape  (3222, 768)

 citance_sentence_embed_arr shape (3222, 768)

 count : 5

 corpus_sentence_embed_arr shape  (3451, 768)

 citance_sentence_embed_arr shape (3451, 768)

 count : 6

 corpus_sentence_embed_arr shape  (2816, 768)

 citance_sentence_embed_arr shape (2816, 768)

 count : 7

 corpus_sentence_embed_arr shape  (3168, 768)

 citance_sentence_embed_arr shape (3168, 768)

 count : 8

In [5]:
import pickle5 as pickle
path = '/ssd_scratch/cvit/dhawals1939/'
with open(path+'updated_test_docs_2018_ngrams_num_15.pkl','rb') as f:
    d = pickle.load(f)
d.keys()

dict_keys(['E03-1005', 'J01-2004', 'W99-0613', 'W06-3114', 'P08-1028', 'A97-1014', 'P08-1043', 'A00-2030', 'D10-1044', 'P04-1036', 'W99-0623', 'P11-1061', 'D09-1092', 'W06-2932', 'A00-2018'])

In [6]:
d['E03-1005'].keys()

dict_keys(['corpus', 'cite_text', 'ref_off', 'corpus_embed', 'cite_text_embed', 'wmd', 'citant_ngram_embed', 'corpus_ngram_embed', 'corpus_ngram_pca_removed3_embed', 'corpus_ngram_pca_removed5_embed', 'citant_ngram_pca_removed3_embed', 'citant_ngram_pca_removed5_embed'])

## pca removal from ngram_2019 documents

In [12]:
import glob
import pickle5 as pickle
path = '/ssd_scratch/cvit/dhawals1939/'
a = glob.glob(path+'test_docs_2019_ngrams*')
# for k in [0,15,19]:
#     file = path+'test_docs_2018_ngrams_num_'+str(k)+'.pkl'
for file in a:
    with open(file,'rb') as f:
        d = pickle.load(f)
    new_file = file.replace('test','updated_test')
    print(new_file)
    create_weightedPCAremoved_ngram(d, des = new_file)


/ssd_scratch/cvit/dhawals1939/updated_test_docs_2019_ngrams_num_150.pkl

 count : 1

 corpus_sentence_embed_arr shape  (1924, 768)

 citance_sentence_embed_arr shape (1924, 768)

 count : 2

 corpus_sentence_embed_arr shape  (2822, 768)

 citance_sentence_embed_arr shape (2822, 768)

 count : 3

 corpus_sentence_embed_arr shape  (2466, 768)

 citance_sentence_embed_arr shape (2466, 768)

 count : 4

 corpus_sentence_embed_arr shape  (2625, 768)

 citance_sentence_embed_arr shape (2625, 768)

 count : 5

 corpus_sentence_embed_arr shape  (2424, 768)

 citance_sentence_embed_arr shape (2424, 768)

 count : 6

 corpus_sentence_embed_arr shape  (3178, 768)

 citance_sentence_embed_arr shape (3178, 768)

 count : 7

 corpus_sentence_embed_arr shape  (2080, 768)

 citance_sentence_embed_arr shape (2080, 768)

 count : 8

 corpus_sentence_embed_arr shape  (640, 768)

 citance_sentence_embed_arr shape (640, 768)

 count : 9

 corpus_sentence_embed_arr shape  (3021, 768)

 citance_sentence_embe


 count : 9

 corpus_sentence_embed_arr shape  (2660, 768)

 citance_sentence_embed_arr shape (2660, 768)

 count : 10

 corpus_sentence_embed_arr shape  (2964, 768)

 citance_sentence_embed_arr shape (2964, 768)

 count : 11

 corpus_sentence_embed_arr shape  (2628, 768)

 citance_sentence_embed_arr shape (2628, 768)

 count : 12

 corpus_sentence_embed_arr shape  (1204, 768)

 citance_sentence_embed_arr shape (1204, 768)

 count : 13

 corpus_sentence_embed_arr shape  (2205, 768)

 citance_sentence_embed_arr shape (2205, 768)

 count : 14

 corpus_sentence_embed_arr shape  (2780, 768)

 citance_sentence_embed_arr shape (2780, 768)

 count : 15

 corpus_sentence_embed_arr shape  (1694, 768)

 citance_sentence_embed_arr shape (1694, 768)
/ssd_scratch/cvit/dhawals1939/updated_test_docs_2019_ngrams_num_165.pkl

 count : 1

 corpus_sentence_embed_arr shape  (3000, 768)

 citance_sentence_embed_arr shape (3000, 768)

 count : 2

 corpus_sentence_embed_arr shape  (432, 768)

 citance_senten


 count : 8

 corpus_sentence_embed_arr shape  (3116, 768)

 citance_sentence_embed_arr shape (3116, 768)

 count : 9

 corpus_sentence_embed_arr shape  (2430, 768)

 citance_sentence_embed_arr shape (2430, 768)

 count : 10

 corpus_sentence_embed_arr shape  (850, 768)

 citance_sentence_embed_arr shape (850, 768)

 count : 11

 corpus_sentence_embed_arr shape  (1083, 768)

 citance_sentence_embed_arr shape (1083, 768)

 count : 12

 corpus_sentence_embed_arr shape  (660, 768)

 citance_sentence_embed_arr shape (660, 768)

 count : 13

 corpus_sentence_embed_arr shape  (3258, 768)

 citance_sentence_embed_arr shape (3258, 768)

 count : 14

 corpus_sentence_embed_arr shape  (850, 768)

 citance_sentence_embed_arr shape (850, 768)

 count : 15

 corpus_sentence_embed_arr shape  (1444, 768)

 citance_sentence_embed_arr shape (1444, 768)
/ssd_scratch/cvit/dhawals1939/updated_test_docs_2019_ngrams_num_180.pkl

 count : 1

 corpus_sentence_embed_arr shape  (1980, 768)

 citance_sentence_em

# n grams new

In [8]:
import glob
import pickle
path = '/ssd_scratch/cvit/dhawals1939/ngrams2/'
a = glob.glob(path+'test_docs_2019_ngrams_withou*')
# for k in [0,15,19]:
#     file = path+'test_docs_2018_ngrams_num_'+str(k)+'.pkl'
for file in a:
    print(file)
    with open(file,'rb') as f:
        unpickler = pickle.Unpickler(f)
        d = unpickler.load()
    new_file = file.replace('test','updated_test')
    print(new_file)
    create_weightedPCAremoved_ngram(d, des = new_file)


/ssd_scratch/cvit/dhawals1939/ngrams2/test_docs_2019_ngrams_withoutsep_num_0.pkl


EOFError: Ran out of input

##  pca removal from ngram_2018_19 documents

In [5]:
import glob
import pickle5 as pickle
path = '/ssd_scratch/cvit/dhawals1939/2018_19/'
a = glob.glob(path+'test_docs_2018_19_ngrams*')
a = sorted(a)

for file in a:
    try:
        with open(file,'rb') as f:
            d = pickle.load(f)
        new_file = file.replace('test','updated_test')
        print(new_file)
        create_weightedPCAremoved_ngram(d, des = new_file)
    except:
        print("error in ",file)


error in  /ssd_scratch/cvit/dhawals1939/2018_19/test_docs_2018_19_ngrams_num_0.pkl
/ssd_scratch/cvit/dhawals1939/2018_19/updated_test_docs_2018_19_ngrams_num_105.pkl

 count : 1

 corpus_sentence_embed_arr shape  (3515, 768)

 citance_sentence_embed_arr shape (3515, 768)

 count : 2

 corpus_sentence_embed_arr shape  (2760, 768)

 citance_sentence_embed_arr shape (2760, 768)

 count : 3

 corpus_sentence_embed_arr shape  (3760, 768)

 citance_sentence_embed_arr shape (3760, 768)

 count : 4

 corpus_sentence_embed_arr shape  (1584, 768)

 citance_sentence_embed_arr shape (1584, 768)

 count : 5

 corpus_sentence_embed_arr shape  (3060, 768)

 citance_sentence_embed_arr shape (3060, 768)

 count : 6

 corpus_sentence_embed_arr shape  (944, 768)

 citance_sentence_embed_arr shape (944, 768)

 count : 7

 corpus_sentence_embed_arr shape  (3402, 768)

 citance_sentence_embed_arr shape (3402, 768)

 count : 8

 corpus_sentence_embed_arr shape  (1134, 768)

 citance_sentence_embed_arr shape 


 count : 11

 corpus_sentence_embed_arr shape  (2120, 768)

 citance_sentence_embed_arr shape (2120, 768)

 count : 12

 corpus_sentence_embed_arr shape  (2538, 768)

 citance_sentence_embed_arr shape (2538, 768)

 count : 13

 corpus_sentence_embed_arr shape  (3116, 768)

 citance_sentence_embed_arr shape (3116, 768)

 count : 14

 corpus_sentence_embed_arr shape  (2430, 768)

 citance_sentence_embed_arr shape (2430, 768)

 count : 15

 corpus_sentence_embed_arr shape  (850, 768)

 citance_sentence_embed_arr shape (850, 768)
error in  /ssd_scratch/cvit/dhawals1939/2018_19/test_docs_2018_19_ngrams_num_225.pkl
/ssd_scratch/cvit/dhawals1939/2018_19/updated_test_docs_2018_19_ngrams_num_236.pkl

 count : 1

 corpus_sentence_embed_arr shape  (2299, 768)

 citance_sentence_embed_arr shape (2299, 768)

 count : 2

 corpus_sentence_embed_arr shape  (1896, 768)

 citance_sentence_embed_arr shape (1896, 768)

 count : 3

 corpus_sentence_embed_arr shape  (1720, 768)

 citance_sentence_embed_arr

In [10]:
import glob
import pickle5 as pickle
path = '/ssd_scratch/cvit/dhawals1939/'
a = glob.glob(path+'test_docs_2018_19_ngrams2*')
a = sorted(a)

for file in a:
    try:
        with open(file,'rb') as f:
            d = pickle.load(f)
        new_file = file.replace('test','updated_test')
        print(new_file)
        create_weightedPCAremoved_ngram(d, des = new_file)
    except:
        print("error in ",file)


/ssd_scratch/cvit/dhawals1939/updated_test_docs_2018_19_ngrams2_num_0.pkl

 count : 1

 corpus_sentence_embed_arr shape  (3341, 768)

 citance_sentence_embed_arr shape (3341, 768)
/ssd_scratch/cvit/dhawals1939/updated_test_docs_2018_19_ngrams2_num_10.pkl

 count : 1

 corpus_sentence_embed_arr shape  (840, 768)

 citance_sentence_embed_arr shape (840, 768)

 count : 2

 corpus_sentence_embed_arr shape  (3179, 768)

 citance_sentence_embed_arr shape (3179, 768)

 count : 3

 corpus_sentence_embed_arr shape  (930, 768)

 citance_sentence_embed_arr shape (930, 768)

 count : 4

 corpus_sentence_embed_arr shape  (1444, 768)

 citance_sentence_embed_arr shape (1444, 768)

 count : 5

 corpus_sentence_embed_arr shape  (4100, 768)

 citance_sentence_embed_arr shape (4100, 768)

 count : 6

 corpus_sentence_embed_arr shape  (1141, 768)

 citance_sentence_embed_arr shape (1141, 768)

 count : 7

 corpus_sentence_embed_arr shape  (1380, 768)

 citance_sentence_embed_arr shape (1380, 768)

 count


 count : 2

 corpus_sentence_embed_arr shape  (2367, 768)

 citance_sentence_embed_arr shape (2367, 768)

 count : 3

 corpus_sentence_embed_arr shape  (3640, 768)

 citance_sentence_embed_arr shape (3640, 768)

 count : 4

 corpus_sentence_embed_arr shape  (1998, 768)

 citance_sentence_embed_arr shape (1998, 768)

 count : 5

 corpus_sentence_embed_arr shape  (10, 768)

 citance_sentence_embed_arr shape (10, 768)

 count : 6

 corpus_sentence_embed_arr shape  (2380, 768)

 citance_sentence_embed_arr shape (2380, 768)

 count : 7

 corpus_sentence_embed_arr shape  (4060, 768)

 citance_sentence_embed_arr shape (4060, 768)

 count : 8

 corpus_sentence_embed_arr shape  (2337, 768)

 citance_sentence_embed_arr shape (2337, 768)

 count : 9

 corpus_sentence_embed_arr shape  (3132, 768)

 citance_sentence_embed_arr shape (3132, 768)

 count : 10

 corpus_sentence_embed_arr shape  (3420, 768)

 citance_sentence_embed_arr shape (3420, 768)
/ssd_scratch/cvit/dhawals1939/updated_test_docs_2